<a href="https://colab.research.google.com/github/wjs20/OAS_data_mining/blob/main/clean_oas_paired.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from pathlib import Path
from tqdm.auto import tqdm
from collections import Counter
import pandas as pd
import re

In [3]:
# download seq files 

sf = """wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Setliff_2019/SRR10313332_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Setliff_2019/SRR10313335_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179273_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179279_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179277_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179287_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179281_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179288_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179295_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179276_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179300_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179299_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179298_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179280_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179292_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179285_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179283_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179274_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179289_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179278_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179297_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179291_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179293_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179275_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179290_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179284_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179294_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179296_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179282_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Goldstein_2019/SRR9179286_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Alsoiussi_2020/SRR11528761_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Alsoiussi_2020/SRR11528762_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/King_2020/BCP006_IgMnegMBC_scVDJ_filtered_contigs_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/King_2020/BCP008_IgMneg_MBC_scVDJ_filtered_contigs_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/King_2020/BCP004_MBC_scVDJ_filtered_contigs_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/King_2020/BCP006_Total_scVDJ_filtered_contigs_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/King_2020/BCP003_Total_scVDJ_filtered_contigs_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/King_2020/BCP009_IgMneg_MBC_scVDJ_filtered_contigs_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/King_2020/BCP009_Total_scVDJ_filtered_contigs_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/King_2020/BCP003_MBC_scVDJ_filtered_contigs_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/King_2020/BCP008_Total_scVDJ_filtered_contigs_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/King_2020/BCP005_IgMnegMBC_scVDJ_filtered_contigs_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/King_2020/BCP004_Total_scVDJ_filtered_contigs_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/King_2020/BCP005_Total_scVDJ_filtered_contigs_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Eccles_2020/SRR10358523_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Eccles_2020/SRR10358524_paired.csv.gz
wget http://opig.stats.ox.ac.uk/webapps/ngsdb/paired_csv/Eccles_2020/SRR10358525_paired.csv.gz
"""

sf = sf.replace("wget ", "")

with open('paired_seq_files.txt', 'wt') as f:
    f.write(sf)

In [4]:
%%capture
!wget -x -nH --cut-dirs=2 -i paired_seq_files.txt

In [5]:
!gunzip -r paired_csv

In [6]:
# format metadata

meta = """Details	study	Filtered Sequences	Organism	Isotype	Chain	Disease	Vaccine	Individual	Age	Longitudinal
Details	Setliff_2019	4103	human	All	Paired	HIV	None	Donor-45	no	Year-17
Details	Setliff_2019	1444	human	All	Paired	HIV	None	Donor-N90	no	Year-23
Details	Goldstein_2019	2287	mouse_BALB/c	All	Paired	None	None	Mouse_1_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	2293	mouse_BALB/c	All	Paired	None	None	Mouse_1_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	3176	rat_SD	All	Paired	None	None	Rat_2_Replicate-2	8-10 weeks	no
Details	Goldstein_2019	1085	rat_SD	All	Paired	None	OVA	Rat_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	3188	rat_SD	All	Paired	None	None	Rat_2_Replicate-2	8-10 weeks	no
Details	Goldstein_2019	1085	rat_SD	All	Paired	None	OVA	Rat_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	2463	rat_SD	All	Paired	None	None	Rat_1_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	3654	rat_SD	All	Paired	None	None	Rat_2_Replicate-2	8-10 weeks	no
Details	Goldstein_2019	3165	rat_SD	All	Paired	None	None	Rat_1_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	3131	rat_SD	All	Paired	None	None	Rat_1_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	3285	rat_SD	All	Paired	None	None	Rat_1_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	2407	mouse_BALB/c	All	Paired	None	None	Mouse_1_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	3305	rat_SD	All	Paired	None	None	Rat_2_Replicate-2	8-10 weeks	no
Details	Goldstein_2019	2111	mouse_BALB/c	All	Paired	None	None	Mouse_2_Replicate-2	8-10 weeks	no
Details	Goldstein_2019	2018	mouse_BALB/c	All	Paired	None	None	Mouse_2_Replicate-2	8-10 weeks	no
Details	Goldstein_2019	2557	mouse_BALB/c	All	Paired	None	None	Mouse_1_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	1070	rat_SD	All	Paired	None	OVA	Rat_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	3389	rat_SD	All	Paired	None	None	Rat_2_Replicate-2	8-10 weeks	no
Details	Goldstein_2019	2956	rat_SD	All	Paired	None	None	Rat_1_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	2400	rat_SD	All	Paired	None	None	Rat_2_Replicate-2	8-10 weeks	no
Details	Goldstein_2019	3457	rat_SD	All	Paired	None	None	Rat_1_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	3942	rat_SD	All	Paired	None	None	Rat_2_Replicate-2	8-10 weeks	no
Details	Goldstein_2019	1117	rat_SD	All	Paired	None	OVA	Rat_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	1700	mouse_BALB/c	All	Paired	None	None	Mouse_2_Replicate-2	8-10 weeks	no
Details	Goldstein_2019	3285	rat_SD	All	Paired	None	None	Rat_1_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	3276	rat_SD	All	Paired	None	None	Rat_1_Replicate-1	8-10 weeks	no
Details	Goldstein_2019	3513	rat_SD	All	Paired	None	None	Rat_2_Replicate-2	8-10 weeks	no
Details	Goldstein_2019	1746	mouse_BALB/c	All	Paired	None	None	Mouse_2_Replicate-2	8-10 weeks	no
Details	Alsoiussi_2020	5742	mouse_C57BL/6	All	Paired	SARS-COV-2	None	no	10 weeks	no
Details	Alsoiussi_2020	5646	mouse_C57BL/6	All	Paired	SARS-COV-2	None	no	10 weeks	no
Details	King_2020	3358	human	All	Paired	Tonsillitis and obstructive sleep apnea	None	BCP6	Year-5	no
Details	King_2020	1075	human	All	Paired	Tonsillitis and obstructive sleep apnea	None	BCP8	Year-3	no
Details	King_2020	2090	human	All	Paired	Tonsillitis	None	BCP4	Year-6	no
Details	King_2020	2812	human	All	Paired	Tonsillitis and obstructive sleep apnea	None	BCP6	Year-5	no
Details	King_2020	1120	human	All	Paired	obstructive sleep apnea	None	BCP3	Year-6	no
Details	King_2020	3390	human	All	Paired	Tonsillitis	None	BCP9	Year-9	no
Details	King_2020	2978	human	All	Paired	Tonsillitis	None	BCP9	Year-9	no
Details	King_2020	2207	human	All	Paired	obstructive sleep apnea	None	BCP3	Year-6	no
Details	King_2020	2888	human	All	Paired	Tonsillitis and obstructive sleep apnea	None	BCP8	Year-3	no
Details	King_2020	5793	human	All	Paired	Tonsillitis	None	BCP5	Year-3	no
Details	King_2020	425	human	All	Paired	Tonsillitis	None	BCP4	Year-6	no
Details	King_2020	2935	human	All	Paired	Tonsillitis	None	BCP5	Year-3	no
Details	Eccles_2020	100	human	All	Paired	None	None	healthy-1	Year-33	no
Details	Eccles_2020	47	human	All	Paired	None	None	healthy-1	Year-33	no
Details	Eccles_2020	624	human	All	Paired	None	None	healthy-1	Year-33	no"""

meta = [l.split('\t') for l in meta.split('\n')]
columns = [s.lower().replace(' ', '_') for s in meta[0]]

In [7]:
# read in csvs and join with correct study metadata
p = Path()/'paired_csv'
csv_files = list(p.glob('**/*.csv'))
dfs = []
for file, meta in tqdm(zip(csv_files, meta[1:]), total=len(csv_files)):
    df = pd.read_csv(file, skiprows=1)
    for k,v in zip(columns, meta):
        df[k] = v
    dfs.append(df)
oas = pd.concat(dfs)

# remove trailing subspecies string from organism series 
orgs = list(oas.organism.unique())
r = re.compile('(?<=\w)_.*')
oas['organism'] = oas.organism.str.replace(r, '')

In [8]:
for animal in oas.organism.unique():
    print(animal, oas.loc[oas['organism'] == animal, 'sequence_alignment_aa_heavy'].nunique())
print('total', oas.sequence_alignment_aa_heavy.nunique())

# number of unique sequences for every species in dataset
    # human 34131
    # mouse 23087
    # rat 51733
    # total 108493

human 34131
mouse 23087
rat 51733
total 108493


In [19]:
import numpy as np

In [61]:
# save data to drive
import json
from pathlib import Path
from collections import defaultdict

aa = list(oas.sequence_alignment_aa_heavy + oas.sequence_alignment_aa_light)
org = list(oas.organism)
org_dict = defaultdict(list)

for org, aa in zip(org, aa):
    org_dict[org].append(aa)

for s in org_dict.keys():
    org_dict[s] = list(set(org_dict[s]))

p = Path()/'drive'/'MyDrive'/'data'
with open(p/'seqs.json', 'w') as f:
    json.dump(org_dict, f)

oas.to_csv(p/'oas.csv')